Chameleon/FABRIC setup

In [ ]:
#General imports
import os
import json
import traceback
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
from datetime import datetime, timedelta
from dateutil import tz
import time

# Chameleon Library
import chi
import chi.lease as lease_manager
import chi.server as server_manager
import chi.network as network_manager

# FABRIC Library
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

In [ ]:
# Change to your project (CHI-XXXXXX) 
chi.set("project_name", "CHI-XXXXXX")

chameleon_prefix =  f'{os.environ["USER"]}_fabric_iperf'
chameleon_server_name = chameleon_prefix+'_server'
chameleon_network_name = chameleon_prefix+'_net'
chameleon_subnet_name = chameleon_prefix+'_subnet'
chameleon_router_name = chameleon_prefix+'_router'
chameleon_lease_name = chameleon_prefix+'_lease'

chameleon_image_name='CC-Ubuntu20.04'
uc_server_count=1
tacc_server_count=1

chameleon_key_name='<Your_Chameleon_Key>'
chameleon_server_reservation_id = '<Your_Chameleon_Server_Reservation>'

In [ ]:
fablib = fablib_manager()
fablib.show_config()

fabric_slice_name='chameleon_stitch'

tacc_site='TACC'
tacc_slice_name="tacc_stitch"
tacc_faciliy_port='Chameleon-TACC'

uc_site='UC'
uc_slice_name="uc_stitch"
uc_faciliy_port='Chameleon-UC'

Make Chameleon reservations

In [ ]:
chi.use_site('CHI@UC')

uc_network_name = chameleon_network_name+'_uc'

# set start/end dates
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date = (datetime.now(tz=tz.tzutc()) + timedelta(hours=1)).strftime(BLAZAR_TIME_FORMAT)

# create lease reservations
uc_reservations = []
lease_manager.add_node_reservation(uc_reservations, count=uc_server_count)
lease_manager.add_fip_reservation(uc_reservations, count=tacc_server_count)
reservation_list.append(
        {
            "resource_type": "network",
            "network_name": uc_network_name,
            "network_properties": "",
            "resource_properties": json.dumps(
                ["==", "$stitch_provider", 'fabric']
            ),
        }
)

# create lease
uc_lease_name = chameleon_lease_name+'_uc'
uc_lease = lease_manager.create_lease(uc_lease_name,
                                     reservations=uc_reservations,
                                     start_date=start_date,
                                     end_date=end_date)

# get lease info
uc_lease_id = lease_manager.get_lease_id(uc_lease_name)
uc_network_reservation_id = [reservation for reservation in uc_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
uc_node_reservation_id = lease_manager.get_node_reservation(uc_lease_id)

In [ ]:
chi.use_site('CHI@TACC')

tacc_network_name = chameleon_network_name+'_tacc'

# set start/end dates
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date = (datetime.now(tz=tz.tzutc()) + timedelta(hours=1)).strftime(BLAZAR_TIME_FORMAT)

# create lease reservations
tacc_reservations = []
lease_manager.add_node_reservation(tacc_reservations, count=tacc_server_count)
lease_manager.add_fip_reservation(tacc_reservations, count=tacc_server_count)
reservation_list.append(
        {
            "resource_type": "network",
            "network_name": tacc_network_name,
            "network_properties": "",
            "resource_properties": json.dumps(
                ["==", "$stitch_provider", 'fabric']
            ),
        }
)

# create lease
tacc_lease_name = chameleon_lease_name+'_tacc'
tacc_lease = lease_manager.create_lease(tacc_lease_name,
                                     reservations=tacc_reservations,
                                     start_date=start_date,
                                     end_date=end_date)

# get lease info
tacc_lease_id = lease_manager.get_lease_id(tacc_lease_name)
tacc_network_reservation_id = [reservation for reservation in tacc_lease['reservations'] if reservation['resource_type'] == 'network'][0]['id']
tacc_node_reservation_id = lease_manager.get_node_reservation(tacc_lease_id)

Check networks

In [ ]:
uc_network_vlan = tacc_network_vlan = None
uc_network_id = tacc_network_id = None
while uc_network_vlan == None or tacc_network_vlan == None:
    try:
        # uc check
        chi.use_site('CHI@UC')
        
        #Get the network
        uc_network = network_manager.get_network(uc_network_name)

        #Get the network ID
        uc_network_id = uc_network['id']
        # print(f'Chameleon Network ID: {chameleon_network_id}')

        #Get the VLAN tag (needed for FABRIC stitching)
        uc_network_vlan = uc_network['provider:segmentation_id']
        # print(f'network_vlan: {network_vlan}')
        
        # tacc check
        chi.use_site('CHI@TACC')
        
        #Get the network
        tacc_network = network_manager.get_network(tacc_network_name)

        #Get the network ID
        tacc_network_id = tacc_network['id']
        # print(f'Chameleon Network ID: {chameleon_network_id}')

        #Get the VLAN tag (needed for FABRIC stitching)
        tacc_network_vlan = tacc_network['provider:segmentation_id']
        # print(f'network_vlan: {network_vlan}')
    except:
        print(f'Chameleon Networks are not ready. Trying again!')
        time.sleep(10) 

FABRIC network slice

In [ ]:
fabric_slice = fablib.new_slice(name=fabric_slice_name)

tacc_facility_port = fabric_slice.add_facility_port(name=tacc_facility_port, site=tacc_site, vlan=str(tacc_network_vlan))
tacc_facility_port_iface = tacc_facility_port.get_interfaces()[0]
uc_facility_port = fabric_slice.add_facility_port(name=uc_facility_port, site=uc_site, vlan=str(uc_network_vlan))
uc_facility_port_iface = uc_facility_port.get_interfaces()[0]

fabric_net = fabric_slice.add_l3network(name=f'facility_port_fabnetv4', interfaces=[tacc_facility_port_iface, uc_facility_port_iface])

In [ ]:
fabric_network = fabric_slice.get_network('facility_port_fabnetv4')

subnet = fabric_network.get_subnet()
fabric_gateway_ip = fabric_network.get_gateway()

available_ips = list(subnet)[1:]
available_ips.remove(fabric_gateway_ip)

chameleon_gateway_ip=available_ips.pop(0)    
chameleon_allocation_pool_start=available_ips[0]
chameleon_allocation_pool_end=available_ips[10]

print(f'fabric_gateway_ip: {fabric_gateway_ip}')
print(f'chameleon_gateway_ip: {chameleon_gateway_ip}')
print(f'chameleon_allocation_pool_start: {chameleon_allocation_pool_start}')
print(f'chameleon_allocation_pool_end: {chameleon_allocation_pool_end}')

Add Chameleon subnets

In [ ]:
chi.use_site('CHI@UC')

uc_subnet_name = chameleon_subnet_name+'_uc'

uc_subnet = network_manager.create_subnet(uc_subnet_name, uc_network_id,
                                         cidr=str(subnet),
                                         allocation_pool_start=chameleon_allocation_pool_start,
                                         allocation_pool_end=chameleon_allocation_pool_end,
                                         gateway_ip=chameleon_gateway_ip)

chi.neutron().update_subnet(subnet=uc_subnet['id'] ,
                                    body={
                                         "subnet": { 
                                             "host_routes": [ 
                                                    {
                                                        "destination": f"{fablib.FABNETV4_SUBNET}", 
                                                         "nexthop": f"{fabric_gateway_ip}"
                                                    }
                                             ] 
                                         }
                                    })

In [ ]:
chi.use_site('CHI@TACC')

tacc_subnet_name = chameleon_subnet_name+'_tacc'

tacc_subnet = network_manager.create_subnet(tacc_subnet_name, tacc_network_id,
                                         cidr=str(subnet),
                                         allocation_pool_start=chameleon_allocation_pool_start,
                                         allocation_pool_end=chameleon_allocation_pool_end,
                                         gateway_ip=chameleon_gateway_ip)

chi.neutron().update_subnet(subnet=tacc_subnet['id'] ,
                                    body={
                                         "subnet": { 
                                             "host_routes": [ 
                                                    {
                                                        "destination": f"{fablib.FABNETV4_SUBNET}", 
                                                         "nexthop": f"{fabric_gateway_ip}"
                                                    }
                                             ] 
                                         }
                                    })

Add Chameleon routers

In [ ]:
chi.use_site('CHI@UC')

uc_router_name = chameleon_router_name+'_uc'

uc_router = network_manager.create_router(uc_router_name, gw_network_name='public')
uc_connection_port = network_manager.add_subnet_to_router_by_name(uc_router_name, uc_subnet_name)

In [ ]:
chi.use_site('CHI@TACC')

tacc_router_name = chameleon_router_name+'_tacc'

tacc_router = network_manager.create_router(tacc_router_name, gw_network_name='public')
tacc_connection_port = network_manager.add_subnet_to_router_by_name(tacc_router_name, tacc_subnet_name)

Start Chameleon servers

In [ ]:
chi.use_site('CHI@UC')

uc_servers = []

for i in range(uc_server_count):
    server_name = f"{uc_server_name}_{i+1}"
    
    servers.append(server_manager.create_server(server_name,
                                               reservation_id=uc_node_reservation_id,
                                               network_name=uc_network_name,
                                               image_name=chameleon_image_name,
                                               key_name=chameleon_key_name
                                            ))

In [ ]:
chi.use_site('CHI@TACC')

tacc_servers = []

for i in range(tacc_server_count):
    server_name = f"{tacc_server_name}_{i+1}"
    
    servers.append(server_manager.create_server(server_name,
                                               reservation_id=tacc_node_reservation_id,
                                               network_name=tacc_network_name,
                                               image_name=chameleon_image_name,
                                               key_name=chameleon_key_name
                                            ))

Wait for servers

In [ ]:
chi.use_site('CHI@UC')

for server in uc_servers:
    server_manager.wait_for_active(server.id)
    
chi.use_site('CHI@TACC')

for server in tacc_servers:
    server_manager.wait_for_active(server.id)

Get server fixed IPs

In [ ]:
chi.use_site('CHI@UC')

uc_fixed_ips={}
for i in range(uc_server_count):
    server_name = f"{uc_server_name}_{i+1}"
    server_id = get_server_id(server_name)
    fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
    uc_fixed_ips[server_name] = fixed_ip

In [ ]:
chi.use_site('CHI@TACC')

tacc_fixed_ips={}
for i in range(tacc_server_count):
    server_name = f"{tacc_server_name}_{i+1}"
    server_id = get_server_id(server_name)
    fixed_ip = get_server(server_id).interface_list()[0].to_dict()["fixed_ips"][0]["ip_address"]
    tacc_fixed_ips[server_name] = fixed_ip